# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [60]:
# import libraries
import re

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'vader_lexicon'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [61]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql('etl', engine)
X = df.message.values
Y = df.iloc[:,4:].copy()

### 2. Write a tokenization function to process your text data

In [62]:
def tokenize(text):
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    words = word_tokenize(text.lower())
    words = [w for w in words if w not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    for w in words:
        lemmatizer.lemmatize(w)
        lemmatizer.lemmatize(w, pos='v')
        lemmatizer.lemmatize(w, pos='a')
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [86]:
pipeline = Pipeline([
    ('nlp', FeatureUnion([
        ('importance', Pipeline([
            ('bow', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
         ('sentiment', VaderSentiment())
        
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [87]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('nlp', FeatureUnion(n_jobs=1,
       transformer_list=[('importance', Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, ma...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [88]:
Y_pred = pipeline.predict(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.84      0.92      0.88      2003
          1       0.78      0.48      0.59       446
          2       0.00      0.00      0.00         7
          3       0.76      0.62      0.68      1082
          4       0.57      0.09      0.16       224
          5       0.80      0.09      0.16       139
          6       1.00      0.03      0.05        77
          7       0.00      0.00      0.00        42
          8       0.75      0.12      0.20       103
          9       0.00      0.00      0.00         0
         10       0.80      0.29      0.43       152
         11       0.84      0.47      0.60       295
         12       0.81      0.37      0.51       228
         13       0.75      0.07      0.12        45
         14       0.33      0.02      0.03        65
         15       0.50      0.03      0.06        29
         16       0.45      0.05      0.09       101
         17       0.71      0.17      0.27   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__n_estimators': [50, 100, 150],
    'clf__max_depth': [3, 6, 9],
    'tfidf__use_idf': [True, False],
    'tfidf__sublinear_tf': [True, False]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = cv.predict(X_test)
print(classification_report(Y_test, Y_pred))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [85]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

class VaderSentiment(BaseEstimator, TransformerMixin):
    
    def __init__(self, tokenizer=None):
        self.vader = SentimentIntensityAnalyzer()
        
    def tokenize(self, text):
        text = re.sub('[^a-zA-Z0-9]', ' ', text)
        words = word_tokenize(text.lower())
        words = [w for w in words if w not in stopwords.words('english')]
        lemmatizer = WordNetLemmatizer()
        for w in words:
            lemmatizer.lemmatize(w)
            lemmatizer.lemmatize(w, pos='v')
            lemmatizer.lemmatize(w, pos='a')
        return words
        
    def analyze(self, text):
        text = ' '.join(self.tokenize(text))
        scores = self.vader.polarity_scores(text)
        return pd.Series([scores['neg'], scores['neu'], scores['pos']])
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        scores = pd.Series(X).apply(self.analyze)
        scores.columns = ['neg', 'neu', 'pos']
        return scores

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.